In [81]:
import os

import numpy as np

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Activation, Dropout, BatchNormalization
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.optimizers import Adam, SGD

from collections import Counter

from sklearn.model_selection import StratifiedKFold, StratifiedShuffleSplit
from sklearn.metrics import accuracy_score

In [82]:
optimizer = "Adam"
loss_function = "categorical_crossentropy"
epochs = 60
batch_size = 128
learning_rate = .01
img_width = 32
img_height = 32
n = 5
num_class = 10
input_shape = (img_width, img_height, 3)

In [83]:
#load CIFAR-10 DATA
def load_data():
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    
    #normalize pixel values to 0-1 to simplify training
    x_train = x_train.astype("float32") / 255.0
    x_test = x_test.astype("float32") / 255.0
    
    #flatten and encode labels
    y_train = y_train.flatten()
    y_test =  y_test.flatten()
    
    return x_train, y_train, x_test, y_test

In [84]:
x_train, y_train, x_test, y_test = load_data()

In [85]:
#AlexNet Model setup - includes matching parameters with MIAShield
def create_model(input_shape, num_class):
    model = Sequential()

    #Layer 1 - Conv2D with MaxPooling
    model.add(Conv2D(48, (3, 3), strides = (2, 2), activation = "relu", padding = "same", input_shape = input_shape))
    model.add(MaxPooling2D(pool_size = (2, 2), strides = (2, 2)))
    model.add(BatchNormalization())

    #Layer 2 - Conv2D with MaxPooling 
    model.add(Conv2D(96, (3, 3), activation = "relu", padding = "same"))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(BatchNormalization())

    #Layer 3 - Conv2D
    model.add(Conv2D(192, (3, 3), activation = "relu", padding = "same"))

    #Layer 4 - Conv2D
    model.add(Conv2D(192, (3, 3), activation = "relu", padding = "same"))

    #Layer 5 - Conv2D with MaxPooling
    model.add(Conv2D(256, (3, 3), activation = "relu", padding = "same"))
    model.add(MaxPooling2D(pool_size = (3, 3), strides = (2, 2)))
    model.add(BatchNormalization())

    #flatten
    model.add(Flatten())

    #Fully Connected layer 1
    model.add(Dense(512, activation = "relu"))
    model.add(Dropout(.50))
            
    #Fully Connected layer 2
    model.add(Dense(256, activation = "relu"))
    model.add(Dropout(.50))

    #Output layer
    model.add(Dense(num_class, activation = "softmax"))

    return model


In [86]:
#base model uses 5 disjoint datasets for training
X = x_train
Y = y_train
idx = np.arange(len(X))

skf = StratifiedKFold(n_splits = n, shuffle = True, random_state = 42)
skf.get_n_splits(X, Y)
    
for i, (_, test_index) in enumerate(skf.split(X,Y)):
    Xi = X[test_index]
    Yi = Y[test_index]
    Y_onehot = to_categorical(Yi, num_classes = num_class)
    index_split = idx[test_index]
    
    globals()[f'x_train_split{i}'] = Xi
    globals()[f'y_train_split{i}'] = Yi
    globals()[f'y_onehot{i}'] = Y_onehot
    globals()[f'index_split{i}'] = index_split

In [87]:
#recombine indices from across all splits
all_indices = np.concatenate([globals()[f'index_split{i}'] for i in range(n)])
print(f"Total number of indices: {len(all_indices)}")

# check for duplicate values to ensure disjointedness
has_duplicates = len(all_indices) != len(np.unique(all_indices))
print("Duplicate Indices Present:", has_duplicates)

# check to ensure full dataset is represented
covers_all = len(all_indices) == len(X)
print("Coverage of Dataset Achieved:", covers_all)

Total number of indices: 50000
Duplicate Indices Present: False
Coverage of Dataset Achieved: True


In [88]:
#verify that label distribution is consistent across all data partitions
for i in range(n):
    labels = globals()[f'y_train_split{i}']
    counts = Counter(labels)
    print(f"Fold {i} label distribution:", dict(counts))

Fold 0 label distribution: {7: 1000, 9: 1000, 4: 1000, 6: 1000, 5: 1000, 0: 1000, 3: 1000, 2: 1000, 1: 1000, 8: 1000}
Fold 1 label distribution: {6: 1000, 9: 1000, 4: 1000, 8: 1000, 7: 1000, 1: 1000, 5: 1000, 3: 1000, 0: 1000, 2: 1000}
Fold 2 label distribution: {9: 1000, 1: 1000, 3: 1000, 6: 1000, 4: 1000, 7: 1000, 2: 1000, 0: 1000, 8: 1000, 5: 1000}
Fold 3 label distribution: {1: 1000, 2: 1000, 3: 1000, 4: 1000, 0: 1000, 9: 1000, 5: 1000, 8: 1000, 6: 1000, 7: 1000}
Fold 4 label distribution: {7: 1000, 2: 1000, 0: 1000, 9: 1000, 3: 1000, 1: 1000, 8: 1000, 5: 1000, 4: 1000, 6: 1000}


In [89]:
#EO uses 2.5k members from each Dtrain split
x_train_sample = []
y_train_sample = []
y_onehot_sample = []
index_original_eo = []
mem_per_split = 2500
nonmem = 5000

for i in range(n):
    Xi = globals()[f'x_train_split{i}']
    Yi = globals()[f'y_train_split{i}']
    y_onehot_i = globals()[f'y_onehot{i}']
    split_original_index = globals()[f'index_split{i}']
    
    stratified_choice = StratifiedShuffleSplit(n_splits = 1, test_size = mem_per_split, random_state = 42)
    
    for _, index_choice in stratified_choice.split(Xi, Yi):
        x_train_sample.append(Xi[index_choice])
        y_train_sample.append(Yi[index_choice])
        y_onehot_sample.append(y_onehot_i[index_choice])
        index_original_eo.extend(split_original_index[index_choice])

#EO uses 5k nonmembers (from Dtest)
stratified_choice_test = StratifiedShuffleSplit(n_splits = 1, test_size = nonmem, random_state = 42)
for _, index_choice_test in stratified_choice_test.split(x_test, y_test):
    x_test_eo = x_test[index_choice_test]
    y_test_eo = y_test[index_choice_test]
    index_test_eo = index_choice_test
    y_test_onehot = to_categorical(y_test_eo, num_classes = num_class)

x_train_eo = np.concatenate(x_train_sample + [x_test_eo], axis = 0)
y_train_eo = np.concatenate(y_train_sample + [y_test_eo], axis = 0)
y_onehot_eo = np.concatenate(y_onehot_sample + [y_test_onehot], axis=0)

members = mem_per_split * n
nonmembers = nonmem
membership_labels = np.concatenate([np.ones(members), np.zeros(nonmem)])

globals()[f'x_train_eo'] = x_train_eo
globals()[f'y_train_eo'] = y_train_eo
globals()[f'y_onehot_eo'] = y_onehot_eo
globals()[f'index_eo_original'] = index_original_eo
globals()[f'index_eo_test'] = index_test_eo
globals()[f'membership_labels_eo'] = membership_labels

In [90]:
# Split back into members and nonmembers to verify breakdowns
members = y_train_eo[:mem_per_split * n]
nonmembers = y_train_eo[mem_per_split * n:]

print(f'Expected Member Count = {n * mem_per_split}')
print(f'Actual Member Count = {len(members)}')
print(f'Expected Nonmember Count = {nonmem}')
print(f'Actual Nonmember Count = {len(nonmembers)}')

# Count stratification of classes and total counts
member_counts = Counter(members)
nonmember_counts = Counter(nonmembers)

print(f'Expected Member Count Per Class = {mem_per_split * n / num_class}')
print(f'Actual Member Count = {member_counts}')
print(f'Expected Nonmember Count Per Class = {nonmem / num_class}')
print(f'Actual Nonmember Count = {nonmember_counts}')


Expected Member Count = 12500
Actual Member Count = 12500
Expected Nonmember Count = 5000
Actual Nonmember Count = 5000
Expected Member Count Per Class = 1250.0
Actual Member Count = Counter({6: 1250, 4: 1250, 3: 1250, 9: 1250, 0: 1250, 2: 1250, 1: 1250, 7: 1250, 5: 1250, 8: 1250})
Expected Nonmember Count Per Class = 500.0
Actual Nonmember Count = Counter({8: 500, 9: 500, 3: 500, 1: 500, 6: 500, 7: 500, 5: 500, 0: 500, 2: 500, 4: 500})


In [91]:
#DTestMiashield uses 5k members from Dtrain - must be disjoint with EO
mems = 5000
nonmems = 5000
x_train_mia = []
y_train_mia = []
y_onehot_mia = []
index_original_mia = []

all_training_indices= np.arange(len(x_train))
remaining_training_indices = np.setdiff1d(all_training_indices, index_eo_original)

for i in range(n):
    Xi = globals()[f'x_train_split{i}']
    Yi = globals()[f'y_train_split{i}']
    y_onehot_i = globals()[f'y_onehot{i}']
    split_original_index = globals()[f'index_split{i}']

    Xi_remaining = Xi[np.isin(split_original_index, np.array(remaining_training_indices))]
    Yi_remaining = Yi[np.isin(split_original_index, np.array(remaining_training_indices))]
    index_remaining = split_original_index[np.isin(split_original_index, np.array(remaining_training_indices))]
    
    stratified_choice = StratifiedShuffleSplit(n_splits = 1, test_size = mems // n, random_state = 42)
    
    for _, index_choice in stratified_choice.split(Xi_remaining, Yi_remaining):
        x_train_mia.append(Xi_remaining[index_choice])
        y_train_mia.append(Yi_remaining[index_choice])
        y_onehot_mia.append(y_onehot_i[index_choice])
        index_original_mia.extend(index_remaining[index_choice])

#DtestMiashield uses 5k nonmembers (from Dtest) - must be disjoinst with EO
all_testing_indices= np.arange(len(x_test))
remaining_testing_indices = np.setdiff1d(all_testing_indices, index_eo_test)

x_test_rem = x_test[remaining_testing_indices]
y_test_rem = y_test[remaining_testing_indices]
index_test_mia = remaining_testing_indices
y_test_onehot = to_categorical(y_test_rem, num_classes = num_class)

x_test_mia = np.concatenate(x_train_mia + [x_test_rem], axis = 0)
y_test_mia = np.concatenate(y_train_mia + [y_test_rem], axis = 0)
y_onehot_mia = np.concatenate(y_onehot_mia + [y_test_onehot], axis=0)

membership_labels = np.concatenate([np.ones(mems), np.zeros(nonmem)])

globals()[f'x_test_mia'] = x_test_mia
globals()[f'y_test_mia'] = y_test_mia
globals()[f'y_onehot_mia'] = y_onehot_mia
globals()[f'index_original_mia'] = index_original_mia
globals()[f'index_test_mia'] = index_test_mia
globals()[f'membership_labels_eo'] = membership_labels


In [92]:
# Split back into members and nonmembers to verify breakdowns
members = y_test_mia[:mems]
nonmembers = y_test_mia[mems:]

print(f'Expected Member Count = {mems}')
print(f'Actual Member Count = {len(members)}')
print(f'Expected Nonmember Count = {nonmems}')
print(f'Actual Nonmember Count = {len(nonmembers)}')

# Count stratification of classes and total counts
member_counts = Counter(members)
nonmember_counts = Counter(nonmembers)

print(f'Expected Member Count Per Class = {mems / num_class}')
print(f'Actual Member Count = {member_counts}')
print(f'Expected Nonmember Count Per Class = {nonmems / num_class}')
print(f'Actual Nonmember Count = {nonmember_counts}')

#check for disjointedness with EO
overlap = np.intersect1d(index_original_eo, index_original_mia)
print(f'Expected overlap = 0')
print(f'Actual overlap = {len(overlap)}')


Expected Member Count = 5000
Actual Member Count = 5000
Expected Nonmember Count = 5000
Actual Nonmember Count = 5000
Expected Member Count Per Class = 500.0
Actual Member Count = Counter({4: 500, 3: 500, 1: 500, 9: 500, 5: 500, 8: 500, 2: 500, 0: 500, 6: 500, 7: 500})
Expected Nonmember Count Per Class = 500.0
Actual Nonmember Count = Counter({0: 500, 6: 500, 3: 500, 1: 500, 9: 500, 8: 500, 7: 500, 5: 500, 4: 500, 2: 500})
Expected overlap = 0
Actual overlap = 0


In [93]:
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range = 10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range = .2,
    zoom_range=0.2,
    horizontal_flip=True,
)

val_datagen = ImageDataGenerator(rescale=1./255)

In [94]:
#standard implementation
for i in range(0, n):
    trained_model = create_model(input_shape, num_class)
    optimizer = tf.keras.optimizers.Adam(learning_rate=.01)
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    trained_model.compile(optimizer = optimizer, loss = loss_function, metrics = ['accuracy'])
    trained_model.summary()
    model_data = globals()[f'x_train_split{i}']
    target_data = globals()[f'y_train_split{i}']
    print(target_data.shape)
    datagen = train_datagen
    steps_per_epoch = len(model_data) // batch_size

    globals()[f'base_history{i}'] = trained_model.fit(datagen.flow(model_data, target_data, batch_size=128, shuffle=True),
                    steps_per_epoch = steps_per_epoch, epochs=epochs, validation_data=(x_test, y_test))
    globals()[f'base_model{i}'] = trained_model

C:\Users\Owner\anaconda3\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_50 (Conv2D)                   │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_30 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_30               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_51 (Conv2D)                   │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_31 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_31               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_52 (Conv2D)                   │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_53 (Conv2D)                   │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_54 (Conv2D)                   │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_32 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_32               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_10 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_30 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_20 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_31 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_21 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_32 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_70 (Conv2D)                   │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_42 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_42               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_71 (Conv2D)                   │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_43 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_43               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_72 (Conv2D)                   │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_73 (Conv2D)                   │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_74 (Conv2D)                   │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_44 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_44               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_14 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_42 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_28 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_43 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_29 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_44 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

(10000,)
Epoch 1/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 9s 73ms/step - accuracy: 0.1374 - loss: 2.7177 - val_accuracy: 0.1124 - val_loss: 2.2947
Epoch 2/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.1562 - loss: 2.1899 - val_accuracy: 0.1000 - val_loss: 2.3072
Epoch 3/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.1801 - loss: 2.1076 - val_accuracy: 0.1161 - val_loss: 2.2809
Epoch 4/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.3047 - loss: 1.9490 - val_accuracy: 0.1267 - val_loss: 2.3912
Epoch 5/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.2231 - loss: 2.0129 - val_accuracy: 0.1103 - val_loss: 2.7674
Epoch 6/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2266 - loss: 1.9318 - val_accuracy: 0.1075 - val_loss: 2.6532
Epoch 7/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.2258 - loss: 1.9942 - val_accuracy: 0.1004 - val_loss: 12.6285
Epoch 8/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.2500 - loss: 1.8645 - val_accuracy: 

In [95]:
import os

save_dir = "models"
os.makedirs(save_dir, exist_ok = True)

for i in range(0, n):
    
    model = globals()[f'base_model{i}']
    
    model_path = os.path.join(save_dir, f'base_model_{i}.h5')
    model.save(model_path)
    print(f"Saved model {i} to {model_path}")


Saved model 0 to models\base_model_0.h5
Saved model 1 to models\base_model_1.h5
Saved model 2 to models\base_model_2.h5
Saved model 3 to models\base_model_3.h5
Saved model 4 to models\base_model_4.h5


In [96]:
# SGD TEST
for i in range(0, n):
    trained_model = create_model(input_shape, num_class)
    optimizer = tf.keras.optimizers.SGD(learning_rate=.01)
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    trained_model.compile(optimizer = optimizer, loss = loss_function, metrics = ['accuracy'])
    trained_model.summary()
    model_data = globals()[f'x_train_split{i}']
    target_data = globals()[f'y_train_split{i}']
    print(target_data.shape)
    datagen = train_datagen
    steps_per_epoch = len(model_data) // batch_size

    globals()[f'sgd_history{i}'] = trained_model.fit(datagen.flow(model_data, target_data, batch_size=128, shuffle=True),
                    steps_per_epoch = steps_per_epoch, epochs=epochs, validation_data=(x_test, y_test))
    globals()[f'sgd_model{i}'] = trained_model

Model: "sequential_15"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_75 (Conv2D)                   │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_45 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_45               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_76 (Conv2D)                   │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_46 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_46               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_77 (Conv2D)                   │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_78 (Conv2D)                   │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_79 (Conv2D)                   │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_47 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_47               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_15 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_45 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_30 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_46 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_31 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_47 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_95 (Conv2D)                   │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_57 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_57               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_96 (Conv2D)                   │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_58 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_58               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_97 (Conv2D)                   │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_98 (Conv2D)                   │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_99 (Conv2D)                   │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_59 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_59               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_19 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_57 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_38 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_58 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_39 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_59 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

(10000,)
Epoch 1/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 7s 74ms/step - accuracy: 0.1392 - loss: 2.2982 - val_accuracy: 0.1471 - val_loss: 2.3015
Epoch 2/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.3281 - loss: 2.0161 - val_accuracy: 0.1474 - val_loss: 2.3015
Epoch 3/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.2466 - loss: 2.0723 - val_accuracy: 0.1273 - val_loss: 2.2972
Epoch 4/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.1953 - loss: 2.0881 - val_accuracy: 0.1336 - val_loss: 2.2969
Epoch 5/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.2804 - loss: 1.9449 - val_accuracy: 0.1585 - val_loss: 2.2996
Epoch 6/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 15ms/step - accuracy: 0.2578 - loss: 1.9698 - val_accuracy: 0.1508 - val_loss: 2.3001
Epoch 7/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 69ms/step - accuracy: 0.3197 - loss: 1.8570 - val_accuracy: 0.1296 - val_loss: 2.6374
Epoch 8/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 17ms/step - accuracy: 0.4219 - loss: 1.7988 - val_accuracy: 0

In [97]:
import os

save_dir = "models"
os.makedirs(save_dir, exist_ok = True)

for i in range(0, n):
    
    model = globals()[f'sgd_model{i}']
    
    model_path = os.path.join(save_dir, f'sgd_model_{i}.h5')
    model.save(model_path)
    print(f"Saved model {i} to {model_path}")


Saved model 0 to models\sgd_model_0.h5
Saved model 1 to models\sgd_model_1.h5
Saved model 2 to models\sgd_model_2.h5
Saved model 3 to models\sgd_model_3.h5
Saved model 4 to models\sgd_model_4.h5


In [98]:
#Adam with change learning rate TEST
for i in range(0, n):
    trained_model = create_model(input_shape, num_class)
    optimizer = tf.keras.optimizers.Adam(learning_rate=.001)
    loss_function = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False)
    trained_model.compile(optimizer = optimizer, loss = loss_function, metrics = ['accuracy'])
    trained_model.summary()
    model_data = globals()[f'x_train_split{i}']
    target_data = globals()[f'y_train_split{i}']
    print(target_data.shape)
    datagen = train_datagen
    steps_per_epoch = len(model_data) // batch_size

    globals()[f'adam_lr001_history{i}'] = trained_model.fit(datagen.flow(model_data, target_data, batch_size=128, shuffle=True),
                    steps_per_epoch = steps_per_epoch, epochs=epochs, validation_data=(x_test, y_test))
    globals()[f'adam_lr001_model{i}'] = trained_model

Model: "sequential_20"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_100 (Conv2D)                  │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_60 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_60               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_101 (Conv2D)                  │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_61 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_61               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_102 (Conv2D)                  │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_103 (Conv2D)                  │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_104 (Conv2D)                  │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_62 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_62               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_20 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_60 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_40 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_61 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_41 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_62 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

...

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ conv2d_120 (Conv2D)                  │ (None, 16, 16, 48)          │           1,344 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_72 (MaxPooling2D)      │ (None, 8, 8, 48)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_72               │ (None, 8, 8, 48)            │             192 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_121 (Conv2D)                  │ (None, 8, 8, 96)            │          41,568 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_73 (MaxPooling2D)      │ (None, 3, 3, 96)            │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_73               │ (None, 3, 3, 96)            │             384 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_122 (Conv2D)                  │ (None, 3, 3, 192)           │         166,080 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_123 (Conv2D)                  │ (None, 3, 3, 192)           │         331,968 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ conv2d_124 (Conv2D)                  │ (None, 3, 3, 256)           │         442,624 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ max_pooling2d_74 (MaxPooling2D)      │ (None, 1, 1, 256)           │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ batch_normalization_74               │ (None, 1, 1, 256)           │           1,024 │
│ (BatchNormalization)                 │                             │                 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ flatten_24 (Flatten)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_72 (Dense)                     │ (None, 512)                 │         131,584 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_48 (Dropout)                 │ (None, 512)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_73 (Dense)                     │ (None, 256)                 │         131,328 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_49 (Dropout)                 │ (None, 256)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_74 (Dense)                     │ (None, 10)                  │           2,570 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 1,250,666 (4.77 MB)

 Trainable params: 1,249,866 (4.77 MB)

 Non-trainable params: 800 (3.12 KB)

(10000,)
Epoch 1/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 10s 71ms/step - accuracy: 0.2108 - loss: 2.1274 - val_accuracy: 0.1003 - val_loss: 2.2944
Epoch 2/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4375 - loss: 1.5365 - val_accuracy: 0.1001 - val_loss: 2.2939
Epoch 3/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 68ms/step - accuracy: 0.3864 - loss: 1.6930 - val_accuracy: 0.1207 - val_loss: 2.2759
Epoch 4/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 13ms/step - accuracy: 0.4219 - loss: 1.6185 - val_accuracy: 0.1348 - val_loss: 2.2757
Epoch 5/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 71ms/step - accuracy: 0.4201 - loss: 1.6102 - val_accuracy: 0.2042 - val_loss: 2.2472
Epoch 6/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 16ms/step - accuracy: 0.4844 - loss: 1.4975 - val_accuracy: 0.2156 - val_loss: 2.2439
Epoch 7/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 6s 70ms/step - accuracy: 0.4608 - loss: 1.4958 - val_accuracy: 0.1819 - val_loss: 2.6988
Epoch 8/60
78/78 ━━━━━━━━━━━━━━━━━━━━ 1s 14ms/step - accuracy: 0.4844 - loss: 1.4832 - val_accuracy: 

In [99]:
import os

save_dir = "models"
os.makedirs(save_dir, exist_ok = True)

for i in range(0, n):
    
    model = globals()[f'adam_lr001_model{i}']
    
    model_path = os.path.join(save_dir, f'adam_lr001_model_{i}.h5')
    model.save(model_path)
    print(f"Saved model {i} to {model_path}")


Saved model 0 to models\adam_lr001_model_0.h5
Saved model 1 to models\adam_lr001_model_1.h5
Saved model 2 to models\adam_lr001_model_2.h5
Saved model 3 to models\adam_lr001_model_3.h5
Saved model 4 to models\adam_lr001_model_4.h5
